# Matrix completion

In [1]:
pip install chompack

     |████████████████████████████████| 8.8 MB 8.2 MB/s eta 0:00:01     |██████▎                         | 1.7 MB 8.2 MB/s eta 0:00:01     |██████████████████████████████  | 8.2 MB 8.2 MB/s eta 0:00:01
     |████████████████████████████████| 12.7 MB 16.5 MB/s eta 0:00:01     |████▊                           | 1.9 MB 16.5 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from cvxopt import matrix, spdiag
import chompack as cp
import numpy as np
import random

## Rules
- $n < N$
- $N' = N + n$ 
- $u < N < u+n$
- $u+n < N'$

## Set properties

In [4]:
random.seed(4242)

N = 100

n_full = [x for x in range(2, N)]
n = random.choice(n_full)

u_full = []
for i in range(N):
    u_full.append(i) if n + i > N else None

N_prime = N + n

In [5]:
print("N : ", N)
print("n : ", n)
print("u : ", u_full)
print("N' : ", N_prime)

N :  100
n :  55
u :  [46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
N' :  155


### Autom

In [119]:
# generate sparse matrix
u = u_full[0]
# to make to stable tests uncomments those lines
N_prime = 7
N = 5
n = 2
u = 4

# different ways of generating the data
## N
N_matrix = matrix(
    [[x+1 for x in range(N)] for x in range(N)]
)
#N_matrix = matrix(
#    [x+1 for x in range(N*N)], (N,N)
#)
for i in range(N):
    N_matrix[i, i] = 8
## n
n_matrix = matrix(
    [[x+1 for x in range(n)] for x in range(n)]
)
#n_matrix = matrix(
#    [x for x in range(n*n)], (n,n)
#)
for i in range(n):
    n_matrix[i, i] = 8
## u
u_matrix = matrix(
    [[1 for x in range(u)] for x in range(u)]
)
#u_matrix = matrix(
#    [x for x in range(u*u)], (u,u)
#)
for i in range(u):
    u_matrix[i, i] = 8

# assembling and add nu matrix
P_matrix = spdiag([N_matrix, n_matrix])
nu_matrix = [x for x in range(N-u, N_prime)]
P_matrix[nu_matrix, nu_matrix] = 10

# in order to have a the diag
for i in range(N_prime):
    P_matrix[i, i] = 8
    
print("N : \n", N_matrix)
print("n : \n", n_matrix)
print("u : \n", u_matrix)

print(P_matrix)

N : 
 [ 8  1  1  1  1]
[ 2  8  2  2  2]
[ 3  3  8  3  3]
[ 4  4  4  8  4]
[ 5  5  5  5  8]

n : 
 [ 8  1]
[ 2  8]

u : 
 [ 8  1  1  1]
[ 1  8  1  1]
[ 1  1  8  1]
[ 1  1  1  8]

[ 8.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00     0         0    ]
[ 2.00e+00  8.00e+00  1.00e+01  1.00e+01  1.00e+01  1.00e+01  1.00e+01]
[ 3.00e+00  1.00e+01  8.00e+00  1.00e+01  1.00e+01  1.00e+01  1.00e+01]
[ 4.00e+00  1.00e+01  1.00e+01  8.00e+00  1.00e+01  1.00e+01  1.00e+01]
[ 5.00e+00  1.00e+01  1.00e+01  1.00e+01  8.00e+00  1.00e+01  1.00e+01]
[    0      1.00e+01  1.00e+01  1.00e+01  1.00e+01  8.00e+00  1.00e+01]
[    0      1.00e+01  1.00e+01  1.00e+01  1.00e+01  1.00e+01  8.00e+00]



In [122]:
def is_semi_pos_def(x):
    return np.all(np.linalg.eigvals(x) >= 0)
def is_semi_pos_def_eigsh(x, epsilon=1e-10):
    return np.all(np.linalg.eigvalsh(x) >= -epsilon)

matrix_tocheck = P_matrix
print(np.linalg.eigvalsh(matrix(matrix_tocheck)))

print(is_semi_pos_def(matrix(matrix_tocheck)))
print(is_semi_pos_def_eigsh(matrix(matrix_tocheck)))

[-3.87935021 -2.         -2.         -2.         -2.          9.22988403
 58.64946618]
False
False


In [63]:
# compute symbolic factorization
symb = cp.symbolic(P_matrix)
#print(symb)

In [116]:
# convert to a chordal sparse matrix 
L = cp.cspmatrix(symb)
L += P_matrix
#print(L)

In [117]:
# make the completion
compl_matrix = cp.psdcompletion(L)
#print(compl_matrix)
print("Finish!")

Finish!
